In [1]:
import pandas as pd
import plotly.express as px
from tqdm import tqdm_notebook as tqdm

import global_utilities as gu
from flights import rapidapi, constants as c, process

rapidapi.tqdm = tqdm
process.tqdm = tqdm

In [2]:
df = rapidapi.query_pair("BCN", "CAG")

In [2]:
df = process.retrive_all_flights()

In [5]:
dbx = gu.dropbox.get_dbx_connector(c.VAR_DROPBOX_TOKEN)

In [8]:
gu.dropbox.write_excel(dbx, df, c.FILE_FLIGHTS, index=False)

In [9]:
df_history = gu.dropbox.read_excel(dbx, c.FILE_FLIGHTS)
df_history

,Price,Direct,Quote_date,Date,Carrier,Origin,Destination,Inserted
0,77,False,2020-01-11 14:36:00,2020-01-14,Ryanair,CAG,GRO,2020-01-12
1,120,False,2020-01-11 14:36:00,2020-01-15,Ryanair,CAG,GRO,2020-01-12
2,135,False,2020-01-11 14:37:00,2020-01-16,Alitalia,CAG,GRO,2020-01-12
3,81,False,2020-01-11 14:37:00,2020-01-17,Ryanair,CAG,GRO,2020-01-12
4,77,False,2020-01-11 14:37:00,2020-01-18,Ryanair,CAG,GRO,2020-01-12
...,...,...,...,...,...,...,...,...
1410,31,True,2020-01-08 18:22:00,2020-10-08,Ryanair,GRO,AHO,2020-01-12
1411,38,True,2020-01-08 18:43:00,2020-10-10,Ryanair,GRO,AHO,2020-01-12
1412,26,True,2020-01-08 09:29:00,2020-10-13,Ryanair,GRO,AHO,2020-01-12
1413,38,True,2020-01-08 18:43:00,2020-10-17,Ryanair,GRO,AHO,2020-01-12


In [18]:
df_out = pd.concat([df_history, df])

In [19]:
df_out.shape

(2830, 8)

In [20]:
cols = [c.COL_DATE, c.COL_QUOTE_DATE, c.COL_ORIGIN, c.COL_DESTINATION, c.COL_DIRECT, c.COL_CARRIER, c.COL_PRICE]

In [21]:
df_out = df_out.drop_duplicates(cols)

In [22]:
df_out.shape

(1415, 8)